In [ ]:
!pip install ta
!pip install yfinance

import json
import requests
import numpy as np
import pandas as pd
import yfinance as yf
from math import exp, log
from tqdm.notebook import tqdm
from datetime import date, timedelta
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
from ta.trend import wma_indicator, CCIIndicator

# Config

In [ ]:
ptype      = 'OHLC4' 		# Price Type = ['Open','High','Low','Close','HL2','OC2','OHL3','HLC3','OHLC4']
reso       = "D"			# ['15', '30', '75', 'D', 'W', 'M']
lookback   = 4       		# Lookback Window Size |2..n|
nlbk       = 3       		# Normalization Lookback |2..240|
lrate      = 0.0009  		# Learning Rate |0.0001..0.01|    minval=0.0001, maxval=0.01, step=0.0001
iterations = 1000    		# Training Iterations |50..20000|
ftype      = 'Volatility'   # Filter Signals by  options=['Volatility','Volume','Both','None'])
valid_ftypes = ['Volatility', 'Volume', 'Both', 'None']
# curves     = (true,  'Show Loss & Prediction Curves?')
# easteregg  = (true,   'Optional Calculation?')  
# useprice   = (true,   'Use Price Data for Signal Generation?')
# holding_p  = (5,      'Holding Period |1..n|',                 minval=1)
sell_strat = "Both" 		# options=["Logistic", "Decisive", "Both"])
use_start_date = True   	# Use start date
startDate = 1
startMonth = 1
startYear = 2021

BUY = 1
SELL = -1
HOLD = 0


# Functions

In [ ]:
def hma(close, period):
	p1 = 2 * wma_indicator(close, period//2)
	p2 = wma_indicator(close, period)
	p3 = p1 - p2
	window = int(period**0.5)
	return wma_indicator(p3, window).round(2)


def rsi(close, period):
	return RSIIndicator(close=close, window=period).rsi().round(2)


def atr(high, low, close, window):
	return AverageTrueRange(high=high, low=low, close=close,
						 window=window).average_true_range().round(2)


def cci(high, low, close, window):
	return CCIIndicator(high=high, low=low, close=close, window=window).round(2).cci()


def volumeBreak(thres, volume):
	rsivol   = rsi(volume, 14)
	osc      = hma(rsivol, 10)
	return osc > thres


def volatilityBreak(high, low, close, volmin, volmax):
	return atr(high, low, close, volmin) > atr(high, low, close, volmax)


def dot(v, w, p):
	prod = v * w
	cumsum = prod.cumsum()
	for i in range(1, len(cumsum) + 1 - p):
		cumsum.iloc[-i] -= cumsum.iloc[-i-p]
	return cumsum

# Fix for series
def minimax(ds, base, p):
	scaled_losses = []
	for i in range(len(ds) - p + 1):
		hi = max(ds.iloc[i : i + p])
		lo = min(ds.iloc[i : i + p])
		highest = max(base.iloc[i : i + p])
		lowest = min(base.iloc[i : i + p])

		scaled_losses.append((highest - lowest) * (ds.iloc[i + p - 1] - lo)/(hi - lo) + lowest)

	return pd.Series(scaled_losses, index=ds.index[p-1:]).round(2)



def sigmoid(z):
	return z.apply(lambda x : 1.0 / (1.0 + exp(-x)))


def get_base_dataset(high, low, close, open):
	if ptype == "Open":
		ds = open
	elif ptype == "High":
		ds = high
	elif ptype == "Low":
		ds = low
	elif ptype == "Close":
		ds = close
	elif ptype == "HL2":
		ds = (high + low)/2
	elif ptype == "OC2":
		ds = (open + close)/2
	elif ptype == "OHL3":
		ds = (open + high + low)/3
	elif ptype == "HLC3":
		ds = (high + low + close)/3
	elif ptype == "OHLC4":
		ds = (open + high + low + close)/4
	else:
		print("Not a valid type.")
		exit(1)

	return ds.round(2)


def get_synthetic_dataset(base_ds):
	return (abs(base_ds ** 2 - 1) + 0.5).apply(np.log).round(2)


def getMomentum(high, low, close, open):
	candleHeight = high - low
	bodyHeight = open - close
	ratio = ((bodyHeight/candleHeight)*100)
	ratio = ratio.abs()
	momentum = ratio > 50
	return momentum


def get_low(momentums, high, low, close, open):
	stop = low.iloc[0]
	stops = [stop]
	for day in range(1, len(momentums)):
		if momentums.iloc[day] and close.iloc[day] - open.iloc[day] > 0:
			temp = low.iloc[day]
			num = 1
			while day - num >= 0 and not momentums.iloc[day-num] and close.iloc[day] > high.iloc[day-num]:
				temp = min(temp, low.iloc[day-num])
				num += 1

			if num > 1:
				stop = temp * 0.99

		stops.append(stop)

	stops = pd.Series(stops, index=momentums.index)

	return stops.round(2)


def getPercentage(reso):
	percentage = 0.0
	if reso == "15":
		percentage = 0.0021
	elif reso == "30":
		percentage = 0.0039
	elif reso == "75":
		percentage = 0.0055
	elif reso == "D":
		percentage = 0.01
	elif reso == "W" or reso == "M":
		percentage = 0.03
	return percentage


def get_buy_signals(base, scaled_loss, filter):
	return (np.logical_and((base.iloc[nlbk-1:] > scaled_loss), filter)).iloc[nlbk-1:]

def get_sell_signals(high, low, close, open, base, scaled_loss, filter):
	momentums = getMomentum(high, low, close, open)
	stop = get_low(momentums, high, low, close, open)

	sell_signals = None
	if sell_strat == "Both":
		logistic = np.logical_and((base.iloc[nlbk-1:] < scaled_loss), filter)
		decisive = close < stop
		sell_signals = (np.logical_or(decisive, logistic))
		sell_signals = sell_signals.iloc[nlbk-1:] # To remove possible NA values
		assert sum(sell_signals.isna()) == 0
	elif sell_strat == "Decisive":
		sell_signals = close < stop
	elif sell_strat == "Logistic":
		sell_signals = np.logical_and((base.iloc[nlbk-1:] < scaled_loss), filter)
	else:
		print("Not a valid sell_start")
		exit(1)

	return sell_signals


#  Take into account decisive as well
def get_all_signals(high, low, close, open, base, scaled_loss, filter):
	buy_signals = get_buy_signals(base, scaled_loss, filter)
	sell_signals = get_sell_signals(high, low, close, open, base, scaled_loss, filter)
	signals = []
	for buy, sell in zip(buy_signals, sell_signals):
		# If sell and buy both, then sell superseeds
		if sell:
			signals.append(SELL)
		elif buy:
			signals.append(BUY)
		else:
			signals.append(HOLD)
	signals = pd.Series(signals, index=buy_signals.index)
	signals = signals[signals != 0]
	signals = signals.diff()
	signals = signals[signals != 0]
	signals.dropna(inplace=True)
	
	return signals/2


# # Might not work
# def logistic_regression(X, Y, p, lr, iterations):
# 	w = 0.0
# 	loss = 0.0
# 	for _ in range(1, iterations + 1):
# 		hypothesis = sigmoid(dot(X, 0.0, p))  # prediction
# 		loss = -1.0 / p * (dot(dot(Y, log(hypothesis) + (1.0 - Y), p), log(1.0 - hypothesis), p))
# 		gradient = 1.0 / p * (dot(X, hypothesis - Y, p))
# 		w = w - lr * gradient                 # update weights
	
# 	return [loss, sigmoid(dot(X, w, p))]             # current loss & prediction

# def logistic_regression(X, Y, p, lr, iterations):
# 	w = 0.0
# 	loss = 0.0
# 	for _ in range(iterations):
# 		hypothesis = sigmoid(dot(X, 0.0, p))  # prediction
# 		chunk1 = dot(Y, hypothesis.apply(np.log) + (1.0 - Y), p)
# 		loss = -1.0 / p * (dot(chunk1, (1.0 - hypothesis).apply(np.log), p))
# 		gradient = 1.0 / p * (dot(X, hypothesis - Y, p))
# 		loss = -1 / p * dot(Y, Y, p)

# 		w = w - lr * gradient                 # update weights
	
# 	gradient = 1.0 / p * (dot(X, 0.5 - Y, p))

# 	return [loss, sigmoid(dot(X, w, p))]             # current loss & prediction



def logistic_regression(X, Y, p):
	w = 0.0
	# Since hypothesis is always 0.5
	chunk1 = dot(Y, (0.3069 - Y), p)
	loss = -1.0 / p * (dot(chunk1, -0.6931, p))

	return [loss, sigmoid(dot(X, w, p))]             # current loss & prediction


# Webhook

In [ ]:
WEBHOOK_URL = DISCORD_WEBHOOK_URL

# message = {"content" : "Testing webhook"}

# payload = {'payload_json': json.dumps(message)}

# r = requests.post(WEBHOOK_URL, payload)

# print(r)

def send_alerts(lastest_date, alert_list):
	alert_list = _beautify(alert_list)
	content = f"Date: {lastest_date}\n\n"
	content += "\n".join(alert_list)
	message = {"content": content}
	payload = {'payload_json': json.dumps(message)}
	r = requests.post(WEBHOOK_URL, payload)
	print(r)


def _beautify(alert_list):
	foo = lambda x : x[:-3]
	alert_list = list(map(foo, alert_list))
	alert_list = list(map(str.capitalize, alert_list))
	return sorted(alert_list)

# Gateway

In [ ]:
FILEPATH = "../input/tradingviewdata/data_dump_0.csv"
FILEPATH_STOCKS = "../input/tradingviewdata/fo_stocks.txt"
FILEPATH_SIGNALS = "../input/tradingviewdata/signals.txt"

def read_data_dump():
	data = pd.read_csv(FILEPATH, header=[0, 1], index_col=0)
	return data


def write_data_dump(data):
	try:
		data.to_csv(FILEPATH, na_rep="NA")
	except:
		print("Permission denied. File may be open.\n")
		return
	
	print("Data successfully written.\n")


def read_stocks_list():
	STOCKS_LIST = []

	with open(FILEPATH_STOCKS, "r") as f:
		STOCKS_LIST = f.readlines()
		STOCKS_LIST = list(map(lambda x: x.strip("\n"), STOCKS_LIST))

	return STOCKS_LIST


def write_signals(output):
	with open(FILEPATH_SIGNALS, mode="w") as f:
		for o in output:
			print(o, file=f)

# Data

In [ ]:
# Get list of all the stocks that have futures
# Stocks excluded- IRCTC.NS, ^NSEBANK, ^NSEI
STOCKS_LIST = read_stocks_list()

# Get completely fresh data
def data_download(sym=STOCKS_LIST, start_date='2020-01-01'):
	if not isinstance(sym, list) and not isinstance(sym, str):
		sym = sym.tolist()

	data = yf.download(sym, start_date)
	if data.empty:
		return None

	data.drop("Adj Close", axis=1, inplace=True)
	data = data.round(decimals=2)
	try:
		data = data.swaplevel(axis=1)
	except:
		pass


	rows = set(data.index)
	data.dropna(inplace=True)
	remaining_rows = set(data.index)
	dropped_rows = rows.difference(remaining_rows)
	print(f"Dropped rows: {[str(d.date()) for d in dropped_rows]}")

	assert data.isnull().any().sum().sum() == 0

	latest_date = str(data.index[-1])
	latest_date = latest_date[:latest_date.index(" ")]

	print(f"Latest date: {latest_date}")

	return data

# Update the existsing csv
def update_data():
	data = read_data_dump()

	latest_date = str(data.index[-1])
	if latest_date.index("-") == 2:
		latest_date = latest_date[6:] + latest_date[2:6] + latest_date[:2]
	
	if latest_date != str(date.today()):
		start_date = str(date.fromisoformat(latest_date) + timedelta(days=1))

		new_data = data_download(STOCKS_LIST, start_date)

		if new_data is None:
			print(f"Data up to date: {latest_date}")
		else:
			new_data.index = new_data.index.date
			updated_df = pd.concat([data, new_data])
			write_data_dump(updated_df)
	else:		
		print(f"Data up to date: {latest_date}")


# Convery to numpy arrays
def get_lists(d):
	volume = np.flip(np.array(d["Volume"]))
	close = np.flip(np.array(d["Close"]))
	open = np.flip(np.array(d["Open"]))
	high = np.flip(np.array(d["High"]))
	low = np.flip(np.array(d["Low"]))

	latest_date = str(d.index.values[-1])
	try:
		latest_date = latest_date[:latest_date.index("T")]
	except:
		pass

	return open, high, low, close, volume, latest_date


In [ ]:
update_data()

# Main

In [ ]:
start_from_date = str(date(startYear, startMonth, startDate))

data = read_data_dump()
data = data.loc[start_from_date:]
latest_date = data.index[-1]
STOCKS_LIST = read_stocks_list()
output = []

print(f"Latest date: {latest_date}\n")

print(f"Price Type: {ptype}")
print(f"Lookback Window: {lookback}")
print(f"Normalization Lookback: {nlbk}")
print(f"Filter Signal: {ftype}")
print(f"Sell strategy: {sell_strat}\n")


for STOCK in tqdm(STOCKS_LIST):
	stock = data[STOCK]

	high = stock["High"]
	low = stock["Low"]
	close = stock["Close"]
	volume = stock["Volume"]
	open = stock["Open"]

	base = get_base_dataset(high, low, close, open)
	synth = get_synthetic_dataset(base)

	[loss, prediction] = logistic_regression(base, synth, lookback)

	scaled_loss = minimax(loss, base, nlbk)

	filter = HOLD
	if ftype == "Volatility":
		filter = volatilityBreak(high, low, close, 1, 10)
	elif ftype == "Volume":
		filter = volumeBreak(49, volume)
	elif ftype == "Both":
		filter = volatilityBreak(high, low, close, 1, 10) and volumeBreak(49, volume)
	else:
		filter = True

	signals = get_all_signals(high, low, close, open, base, scaled_loss, filter)

	if signals.index[-1] == latest_date:
		if signals.iloc[-1] == BUY:
			output.append(f"BUY {STOCK}")
		elif signals.iloc[-1] == SELL:
			output.append(f"SELL {STOCK}")
		else:
			output.append(f"Not a valid signal {STOCK}")
	
for o in output:
	print(o)

send_alerts(latest_date, output)